<a href="https://colab.research.google.com/github/aviraltyagi/GenderDetection/blob/master/GenderDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.applications import InceptionV3, VGG16, ResNet50, VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from glob import glob
import typing

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content'

In [ ]:
!kaggle datasets download -d cashutosh/gender-classification-dataset

 95% 257M/269M [00:03<00:00, 69.4MB/s]
100% 269M/269M [00:03<00:00, 83.4MB/s]


In [ ]:
!unzip \*.zip && rm *.zip

In [ ]:
train_dataset = "/content/Training"
validation_dataset = "/content/Validation"

train_folders = glob("/content/Training/*")
print(train_folders)

['/content/Training/female', '/content/Training/male']


In [ ]:
train_datagen = ImageDataGenerator(rescale= 1./255)
test_datagen = ImageDataGenerator(rescale= 1./255)

train_set = train_datagen.flow_from_directory(train_dataset, target_size= (114, 92), batch_size= 64, class_mode= 'categorical')
test_set = test_datagen.flow_from_directory(train_dataset, target_size= (114, 92), batch_size= 64, class_mode= 'categorical')

Found 47009 images belonging to 2 classes.
Found 47009 images belonging to 2 classes.


In [ ]:
def CustomModel(input_shape: typing.Tuple[int]) -> Model:
    vgg19 = VGG19(input_shape= input_shape, weights= 'imagenet', include_top= False)
    for layer in vgg19.layers:
        layer.trainable = False
        
    x = Flatten()(vgg19.output)
    x = Dense(units= 2048, activation= 'relu')(x)
    output = Dense(units= len(train_folders), activation= 'softmax')(x)
    
    model = Model(inputs= vgg19.input, outputs= output, name= 'EmotionDetection')
    model.compile(loss= 'categorical_crossentropy', optimizer= 'adam', metrics= ['accuracy'])
    
    return model

model = CustomModel(input_shape= (114,92,3))
history = model.fit(train_set,
                    validation_data= test_set,
                    epochs= 2,
                    callbacks= [tf.keras.callbacks.EarlyStopping(patience=5, monitor='val_loss',restore_best_weights=True)],
                    steps_per_epoch= len(train_set),
                    validation_steps= len(test_set))

Epoch 1/2
735/735 [==============================] - 101s 137ms/step - loss: 0.4697 - accuracy: 0.8497 - val_loss: 0.2055 - val_accuracy: 0.9167
Epoch 2/2
735/735 [==============================] - 109s 148ms/step - loss: 0.2006 - accuracy: 0.9206 - val_loss: 0.1887 - val_accuracy: 0.9247


In [ ]:
# def CustomModel(input_shape: typing.Tuple[int]) -> Model:
#     inceptionV3 = InceptionV3(input_shape= input_shape, weights= 'imagenet', include_top= False)
#     for layer in inceptionV3.layers:
#         layer.trainable = False
        
#     x = Flatten()(inceptionV3.output)
#     x = Dense(units= 2048, activation= 'relu')(x)
#     output = Dense(units= len(train_folders), activation= 'softmax')(x)
    
#     model = Model(inputs= inceptionV3.input, outputs= output, name= 'EmotionDetection')
#     model.compile(loss= 'categorical_crossentropy', optimizer= 'adam', metrics= ['accuracy'])
    
#     return model

In [ ]:
# model = CustomModel(input_shape= (114,92,3))
# model.summary()

In [ ]:
# history = model.fit(
#                     train_set,
#                     validation_data= test_set,
#                     epochs= 10,
#                     callbacks= [tf.keras.callbacks.EarlyStopping(patience=5, monitor='val_loss',restore_best_weights=True)],
#                     steps_per_epoch= len(train_set),
#                     validation_steps= len(test_set))

In [ ]:
model.save("/content/drive/MyDrive/Colab Data/AgeGender/GenderModelV19.h5")
model_yaml = model.to_yaml()
with open("/content/drive/MyDrive/Colab Data/AgeGender/yamlmodel.yaml", "w") as yaml_file:
  yaml_file.write(model_yaml)
model.save_weights("/content/drive/MyDrive/Colab Data/AgeGender/weight.h5")